Task 1: Extract main data from 2 data sources

In [43]:
import pandas as pd
import requests

url = "https://raw.githubusercontent.com/Papagoat/brain-assessment/main/restaurant_data.json"
response = requests.get(url)
restaurant_data = response.json()
country_path = 'Country-Code.xlsx'
country_df =   pd.read_excel(country_path)


Task 1 Part 1: Extract restaurant data into data frame and combine with country dataframe 

In [44]:
restaurants_list = []

for entry in restaurant_data:
    for restaurant in entry['restaurants']:
        info = {
            'Restaurant Id': restaurant['restaurant']['R']['res_id'],
            'Restaurant Name': restaurant['restaurant']['name'],
            'Country Code': restaurant['restaurant']['location']['country_id'], 
            'City': restaurant['restaurant']['location']['city'],
            'User Rating Votes': restaurant['restaurant']['user_rating']['votes'],
            'User Aggregate Rating': float(restaurant['restaurant']['user_rating']['aggregate_rating']),
            'Cuisines': restaurant['restaurant']['cuisines']
        }
        restaurants_list.append(info)
restaurants_df = pd.DataFrame(restaurants_list)
combined_df = pd.merge(restaurants_df, country_df, on='Country Code', how='left')
final_df = combined_df.drop('Country Code', axis=1)
final_df.to_csv('restaurants.csv', index=False)


Task 1 Part 2: Extract list of restaurants with events in April 2019 by checking event timeframe

In [45]:
from datetime import datetime

event_list = []

start_date = datetime(2019, 4, 1)
end_date = datetime(2019, 4, 30)

for entry in restaurant_data:
    for restaurant in entry['restaurants']:
        if 'zomato_events' in restaurant['restaurant']:
            for event in restaurant['restaurant']['zomato_events']:
                event_start = datetime.strptime(event['event']['start_date'], '%Y-%m-%d')
                event_end = datetime.strptime(event['event']['end_date'], '%Y-%m-%d')
                if start_date <= event_start <= end_date or start_date <= event_end <= end_date:
                    if 'photos' in event['event'] and len(event['event']['photos']) > 0:
                        photo_url = event['event']['photos'][0]['photo']['url']
                    else:
                        photo_url = "NA"
                    info = {
                        'Event Id': event['event']['event_id'],
                        'Restaurant Id': restaurant['restaurant']['R']['res_id'],
                        'Restaurant Name': restaurant['restaurant']['name'],
                        'Photo URL': photo_url,
                        'Event Title': event['event']['title'],
                        'Event Start Date': event['event']['start_date'],
                        'Event End Date': event['event']['end_date']
                    }
                    event_list.append(info)

event_df = pd.DataFrame(event_list)

event_df.fillna("NA", inplace=True)

event_df.to_csv('restaurant_events.csv', index=False)

Task 2 Part 3: Determine rating text based on threshholds

In [46]:
rating_categories = {
    'Excellent': [],
    'Very Good': [],
    'Good': [],
    'Average': [],
    'Poor': []
}

for entry in restaurant_data:
    for restaurant in entry['restaurants']:
        text = restaurant['restaurant']['user_rating']['rating_text']
        rating = float(restaurant['restaurant']['user_rating']['aggregate_rating'])
        if text in rating_categories:
            rating_categories[text].append(rating)

for category in rating_categories:
    rating_categories[category] = sorted(list(set(rating_categories[category])))

for category, ratings in rating_categories.items():
    print(f"{category}: {ratings}")

Excellent: [4.5, 4.6, 4.7, 4.8, 4.9]
Very Good: [4.0, 4.1, 4.2, 4.3, 4.4]
Good: [3.5, 3.6, 3.7, 3.8, 3.9]
Average: [2.5, 2.6, 2.7, 2.8, 2.9, 3.0, 3.1, 3.2, 3.3, 3.4]
Poor: [2.2]
